In [1]:
import numpy as np

In [37]:
# Lecture 1
def call_option_payoff(stock_price_at_expiration, strike_price):
    """
    Computes the payoff of a call option at expiration.
    
    Parameters:
    - stock_price_at_expiration (float): The price of the underlying stock at the option's expiration.
    - strike_price (float): The strike price of the call option.
    
    Returns:
    - float: The payoff of the call option, which is max(stock price at expiration - strike price, 0).
    """
    return max(stock_price_at_expiration - strike_price, 0)

def put_option_payoff(stock_price_at_expiration, strike_price):
    """
    Computes the payoff of a put option at expiration.
    
    Parameters:
    - stock_price_at_expiration (float): The price of the underlying stock at the option's expiration.
    - strike_price (float): The strike price of the put option.
    
    Returns:
    - float: The payoff of the put option, which is max(strike price - stock price at expiration, 0).
    """
    return max(strike_price - stock_price_at_expiration, 0)


def put_call_parity(S0, K, r, T, C=None, P=None, D=0):
    """
    General Put-Call Parity function, adaptable for cases with or without dividends.
    
    Parameters:
    - S0: Current stock price
    - K: Strike price
    - r: Risk-free interest rate (%)
    - T: Time to expiration (days)
    - C: Call option price (if known)
    - P: Put option price (if known)
    - D: Dividends (default is 0 for no dividends)
    
    Returns:
    - Computed value or verification based on the inputs.
    """
    r = r/100
    T = T/365
    PV_K = K * np.exp(-r * T)  # Present value of strike price
    
    if C is not None and P is not None:
        lhs = C - P
        rhs = S0 - PV_K - D
        if lhs > rhs:
            # There's an arbitrage: Buy put, sell call, buy stock, borrow PV(K)
            arbitrage_profit = lhs - rhs
            action = "Buy put, sell call, buy stock, and borrow the present value of the strike price."
        elif lhs < rhs:
            # There's an arbitrage: Sell put, buy call, short stock, lend PV(K)
            arbitrage_profit = rhs - lhs
            action = "Sell put, buy call, short sell the stock, and lend the present value of the strike price."
        else:
            return "No arbitrage opportunity exists."
        return f"Arbitrage opportunity: {action} Expected profit: {arbitrage_profit:.2f}"    
    elif C is not None:
        return C - S0 + PV_K + D  # Computes P
    elif P is not None:
        return P + S0 - PV_K - D  # Computes C
    else:
        raise ValueError("Either C or P must be provided.")

def annualize_rate(monthly_rate):
    """
    Annualizes a monthly interest rate.
    
    Parameters:
    - monthly_rate: The monthly rate as a decimal (e.g., 1% = 0.01)
    
    Returns:
    - The annualized rate.
    """
    return (1 + monthly_rate)**12 - 1

def put_call_parity_continuous_dividends(S0, K, r, q, T, C=None, P=None):
    """
    Computes the Put-Call Parity for options on stocks that pay continuous dividends.
    
    Parameters:
    - S0: Current stock price
    - K: Strike price
    - r: Risk-free interest rate (annualized)
    - q: Continuous dividend yield (annualized)
    - T: Time to expiration (in years)
    - C: Price of the European call option (if known)
    - P: Price of the European put option (if known)
    
    Returns:
    - The computed value based on the provided information.
    """
    r = r/100
    T = T/365
    PV_K = K * np.exp(-r * T)  # Present value of strike price
    adjusted_S0 = S0 * np.exp(-q * T)  # Adjusted stock price for continuous dividends

    if C is not None and P is not None:
        print("Both call and put prices are provided. Verifying Put-Call Parity for continuous dividends.")
        lhs = C - P
        rhs = adjusted_S0 - PV_K
        return lhs, rhs
    elif C is not None:
        return C - adjusted_S0 + PV_K  # Computes P
    elif P is not None:
        return P + adjusted_S0 - PV_K  # Computes C
    else:
        raise ValueError("Either C or P must be provided.")

In [20]:
# Lecture 2
def find_arbitrage_opportunity(K1, K2, C1, C2, P1, P2):
    """
    Identifies and suggests arbitrage strategies for given call and put prices with different strike prices.
    
    Parameters:
    - K1, K2: Strike prices where K1 < K2.
    - C1, C2: Prices of call options for K1 and K2 respectively.
    - P1, P2: Prices of put options for K1 and K2 respectively.
    
    Returns:
    - Suggestions for arbitrage opportunities.
    """
    call_spread = C1 - C2
    put_spread = P2 - P1
    strike_difference = K2 - K1
    
    strategies = []
    
    # Check for call spread arbitrage
    if call_spread > strike_difference:
        strategies.append(f"Buy call with K2={K2}, sell call with K1={K1}.")
    
    # Check for put spread arbitrage
    if put_spread > strike_difference:
        strategies.append(f"Buy put with K1={K1}, sell put with K2={K2}.")
    
    if strategies:
        return "Arbitrage opportunity: " + " AND ".join(strategies)
    else:
        return "No clear arbitrage opportunity detected."

In [3]:
# Week 2
def call_option_value_risk_neutral(S0, K, S_up, S_down):
    # Calculate the risk-neutral probability
    p_rn = (S0 - S_down) / (S_up - S_down)
    
    # Calculate the payoffs in each scenario
    payoff_up = max(S_up - K, 0)
    payoff_down = max(S_down - K, 0)
    
    # Calculate the expected payoff using risk-neutral probabilities
    expected_payoff_rn = (payoff_up * p_rn) + (payoff_down * (1 - p_rn))
    
    # Present value is equal to the expected payoff when the riskless rate is zero
    present_value_rn = expected_payoff_rn
    
    return present_value_rn

In [5]:
S0 = 100  # Current stock price
K = 110  # Strike price
S_up = 130  # Stock price if it goes up
S_down = 70  # Stock price if it goes down
p_up = 0.80  # Probability of going up
p_down = 0.20  # Probability of going down

# Calculate the value of the call option using risk-neutral probabilities
option_value_rn = call_option_value_risk_neutral(S0, K, S_up, S_down)
option_value_rn

10.0

In [4]:
# Binomial Tree Fast using vectorising
def binomial_tree_fast(K, T, S0, r, N, u, d, sigma=0,opttype='C'):
    dt = T / N
    q = (np.exp((r-sigma) * dt) - d) / (u - d)
    disc = np.exp(-r * dt)
    
    print(q)
    # Price at each node at maturity
    C = S0 * d ** np.arange(N, -1, -1) * u ** np.arange(0, N + 1, 1)
    
    if opttype == 'C':
        # Calculate payoffs for a call option at maturity
        C = np.maximum(C - K, np.zeros(N + 1))
    elif opttype == 'P':
        # Calculate payoffs for a put option at maturity
        C = np.maximum(K - C, np.zeros(N + 1))
      
    print(C)
    
    # Backward induction through the tree
    for i in range(N, 0, -1):
        C = disc * (q * C[1:i + 1] + (1 - q) * C[0:i])
    
    return C[0]

In [5]:
S0 = 100      # Initial Stock Price
K = 200         # Strike Price
T = 1       # Time to maturity in years
r = 0.06       # Annual risk free rate
sigma = 0.03
std = 0.30
N = 9          # Number of time steps
u = 1.3            # Up-factor in binomial model
d = 0.7       # Ensure recombining Tree
opptype='P'    # Option Type 'C' or 'P'
binomial_tree_fast(K, T, S0, r, N, u, d, sigma , opptype)

0.5055648251114596
[195.9646393 192.5057587 186.0821233 174.1525147 151.9975273 110.8525507
  34.4404513   0.          0.          0.       ]


107.21201393677765

In [ ]:
NP.ARRA

In [38]:
np.exp(0.01*0.75 + (0.30 * np.sqrt(0.75)))

1.3064422737776826

In [32]:
1/np.exp(0.03-0.02) * 0.75 - (0.30 * 0.75 ** 1/2)

0.630037375311876